# Import data

In [ ]:
from google.colab import files
uploaded = files.upload()


Saving train.En.csv to train.En (1).csv


In [ ]:
from google.colab import files
uploaded = files.upload()

Saving task_A_En_test.csv to task_A_En_test (1).csv


In [ ]:
import pandas as pd

data_train_en = pd.read_csv('train.En.csv')
data_test_A_en = pd.read_csv('task_A_En_test.csv')

In [ ]:
#There is null in train dataset
data_train_en = data_train_en.dropna(subset=['tweet'])

# BERT word embding and the result of SVM

In [ ]:
import torch
from transformers import BertTokenizer, BertModel

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

In [ ]:
texts = data_train_en['tweet'].tolist()
y_train = data_train_en['sarcastic'].tolist()
inputs = tokenizer(texts, padding=True, truncation=True, return_tensors="pt", max_length=512)
with torch.no_grad():
    outputs = model(**inputs)
    X_train = outputs.last_hidden_state[:,0,:].numpy()

In [ ]:
texts = data_test_A_en['text'].tolist()
y_test = data_test_A_en['sarcastic'].tolist()
inputs = tokenizer(texts, padding=True, truncation=True, return_tensors="pt", max_length=512)
with torch.no_grad():
    outputs = model(**inputs)
    X_test = outputs.last_hidden_state[:,0,:].numpy()

In [ ]:
#train svm and the result
from sklearn.svm import SVC
from sklearn.metrics import classification_report

svm_model = SVC(kernel='linear')
svm_model.fit(X_train, y_train)
predictions = svm_model.predict(X_test)
print(classification_report(y_test, predictions))

              precision    recall  f1-score   support

           0       0.88      0.81      0.84      1200
           1       0.22      0.34      0.27       200

    accuracy                           0.74      1400
   macro avg       0.55      0.57      0.56      1400
weighted avg       0.79      0.74      0.76      1400



# Bert Starts

In [ ]:
from transformers import BertForSequenceClassification
from torch.utils.data import DataLoader, RandomSampler, TensorDataset, SequentialSampler


model_name = 'bert-base-uncased'
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertForSequenceClassification.from_pretrained(model_name, num_labels=2)


texts_train = data_train_en['tweet'].tolist()
y_train = data_train_en['sarcastic'].tolist()
train_encodings = tokenizer(texts_train, padding=True, truncation=True, return_tensors="pt", max_length=512)
train_labels = torch.tensor(y_train)

texts_test = data_test_A_en['text'].tolist()
y_test = data_test_A_en['sarcastic'].tolist()
test_encodings = tokenizer(texts_test, padding=True, truncation=True, return_tensors="pt", max_length=512)
test_labels = torch.tensor(y_test)


train_dataset = TensorDataset(train_encodings['input_ids'], train_encodings['attention_mask'], train_labels)
test_dataset = TensorDataset(test_encodings['input_ids'], test_encodings['attention_mask'], test_labels)


batch_size = 16
train_dataloader = DataLoader(train_dataset, sampler=RandomSampler(train_dataset), batch_size=batch_size)
test_dataloader = DataLoader(test_dataset, sampler=SequentialSampler(test_dataset), batch_size=batch_size)


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)






Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

In [ ]:
from tqdm import tqdm
import numpy as np

optimizer = torch.optim.AdamW(model.parameters(), lr=2e-5)

epochs = 3
for epoch in range(epochs):

    model.train()
    train_loss = 0
    train_correct = 0
    total = 0
    train_progress_bar = tqdm(train_dataloader, desc=f'Epoch {epoch+1}/{epochs}', leave=True)

    for batch in train_progress_bar:
        batch = tuple(b.to(device) for b in batch)
        inputs = {'input_ids': batch[0], 'attention_mask': batch[1], 'labels': batch[2]}

        model.zero_grad()
        outputs = model(**inputs)
        loss = outputs.loss
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        logits = outputs.logits
        predictions = torch.argmax(logits, dim=-1)
        total += batch[2].size(0)
        train_correct += (predictions == batch[2]).sum().item()

        train_progress_bar.set_postfix({'Training Loss': '{:.3f}'.format(loss.item())})

    avg_train_loss = train_loss / len(train_dataloader)
    train_accuracy = train_correct / total
    print(f"\nAverage Training Loss: {avg_train_loss:.2f}")
    print(f"Training Accuracy: {train_accuracy:.2f}")





Epoch 1/3: 100%|██████████| 217/217 [00:24<00:00,  8.96it/s, Training Loss=0.516]



Average Training Loss: 0.57
Training Accuracy: 0.74


Epoch 2/3: 100%|██████████| 217/217 [00:23<00:00,  9.41it/s, Training Loss=0.652]



Average Training Loss: 0.55
Training Accuracy: 0.75


Epoch 3/3: 100%|██████████| 217/217 [00:23<00:00,  9.41it/s, Training Loss=0.332]


Average Training Loss: 0.41
Training Accuracy: 0.81


In [ ]:
model.eval()
all_predictions = []
all_true_labels = []

for batch in tqdm(test_dataloader, desc='Final Evaluation'):
    batch = tuple(b.to(device) for b in batch)
    inputs = {'input_ids': batch[0], 'attention_mask': batch[1]}

    with torch.no_grad():
        outputs = model(**inputs)

    logits = outputs.logits
    predictions = torch.argmax(logits, dim=-1)
    all_predictions.extend(predictions.cpu().numpy())
    all_true_labels.extend(batch[2].cpu().numpy())

report = classification_report(all_true_labels, all_predictions, target_names=['Class 0', 'Class 1'], digits=2, output_dict=False)
print(report)


Final Evaluation: 100%|██████████| 88/88 [00:03<00:00, 25.85it/s]

              precision    recall  f1-score   support

     Class 0       0.88      0.87      0.87      1200
     Class 1       0.26      0.28      0.27       200

    accuracy                           0.78      1400
   macro avg       0.57      0.57      0.57      1400
weighted avg       0.79      0.78      0.79      1400

